# Part 3.3: PyTorch Fundamentals

You've built neural networks from scratch using NumPy. You understand backpropagation, gradient descent, and the chain rule at a fundamental level. Now it's time to use a framework that handles the tedious parts automatically while giving you full control over the model architecture.

**PyTorch** is the framework of choice for research and increasingly for production. It feels like NumPy with superpowers: automatic differentiation, GPU acceleration, and a rich ecosystem of tools.

---

## Learning Objectives

By the end of this notebook, you should be able to:

- [ ] Explain why PyTorch is preferred over raw NumPy for deep learning
- [ ] Create and manipulate tensors (PyTorch's version of arrays)
- [ ] Understand autograd and how it tracks gradients automatically
- [ ] Build custom neural network modules using nn.Module
- [ ] Implement a complete training loop from scratch
- [ ] Save and load model weights

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
torch.manual_seed(42)
np.random.seed(42)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

---

## 1. Why PyTorch?

### The Problem with From-Scratch Implementations

Remember building backpropagation by hand? You had to:
1. Manually compute gradients for every operation
2. Carefully chain them together using the chain rule
3. Keep track of intermediate values for the backward pass
4. Run everything on CPU (slow for large models)

This was valuable for understanding, but it doesn't scale. Modern networks have millions of parameters and complex architectures. PyTorch solves this.

### NumPy vs PyTorch Comparison

| Feature | NumPy | PyTorch |
|---------|-------|--------|
| **Array/Tensor creation** | `np.array([1, 2, 3])` | `torch.tensor([1, 2, 3])` |
| **Random arrays** | `np.random.randn(3, 4)` | `torch.randn(3, 4)` |
| **Matrix multiply** | `a @ b` or `np.dot(a, b)` | `a @ b` or `torch.mm(a, b)` |
| **Element-wise ops** | `a + b`, `a * b` | `a + b`, `a * b` |
| **Automatic gradients** | No | Yes (autograd) |
| **GPU support** | No | Yes |
| **Neural network layers** | No | Yes (nn.Module) |

**Key insight:** PyTorch syntax is nearly identical to NumPy, but with superpowers.

### The Three Superpowers of PyTorch

| Superpower | What it does | Why it matters |
|------------|--------------|----------------|
| **Autograd** | Automatically computes gradients | No more manual chain rule! |
| **GPU Acceleration** | Runs computations on graphics cards | 10-100x faster training |
| **Ecosystem** | Pre-built layers, optimizers, datasets | Focus on research, not plumbing |

In [ ]:
# Quick comparison: NumPy vs PyTorch syntax
print("=" * 50)
print("NumPy vs PyTorch: Nearly identical syntax!")
print("=" * 50)

# NumPy
np_array = np.array([1.0, 2.0, 3.0])
print(f"\nNumPy array:  {np_array}")
print(f"NumPy type:   {type(np_array)}")

# PyTorch
torch_tensor = torch.tensor([1.0, 2.0, 3.0])
print(f"\nPyTorch tensor: {torch_tensor}")
print(f"PyTorch type:   {type(torch_tensor)}")

# Same operations!
print(f"\nNumPy sum:    {np_array.sum()}")
print(f"PyTorch sum:  {torch_tensor.sum()}")

print(f"\nNumPy mean:   {np_array.mean()}")
print(f"PyTorch mean: {torch_tensor.mean()}")

---

## 2. Tensors: The Foundation

### What is a Tensor?

A tensor is just a multi-dimensional array - PyTorch's equivalent of NumPy's `ndarray`. The name comes from physics/math, but for our purposes:

- **0D tensor (scalar):** A single number
- **1D tensor (vector):** A list of numbers
- **2D tensor (matrix):** A table of numbers
- **3D+ tensor:** Multiple matrices stacked (images, batches, etc.)

### Creating Tensors

| Method | Example | Use case |
|--------|---------|----------|
| From data | `torch.tensor([1, 2, 3])` | Convert existing data |
| Zeros | `torch.zeros(3, 4)` | Initialize biases |
| Ones | `torch.ones(3, 4)` | Create masks |
| Random normal | `torch.randn(3, 4)` | Initialize weights |
| Random uniform | `torch.rand(3, 4)` | Values in [0, 1) |
| Range | `torch.arange(0, 10, 2)` | Create sequences |
| Like another | `torch.zeros_like(x)` | Match shape/type |

In [ ]:
# Creating tensors - many ways!
print("=" * 50)
print("Creating Tensors")
print("=" * 50)

# From Python data
from_list = torch.tensor([1, 2, 3, 4])
print(f"\nFrom list: {from_list}")

from_nested = torch.tensor([[1, 2], [3, 4], [5, 6]])
print(f"From nested list:\n{from_nested}")

# Initialized tensors
zeros = torch.zeros(2, 3)
print(f"\nZeros (2x3):\n{zeros}")

ones = torch.ones(3, 2)
print(f"\nOnes (3x2):\n{ones}")

# Random tensors (most common for weight initialization)
random_normal = torch.randn(2, 3)  # Standard normal distribution
print(f"\nRandom normal:\n{random_normal}")

random_uniform = torch.rand(2, 3)  # Uniform [0, 1)
print(f"\nRandom uniform:\n{random_uniform}")

# Range-based
sequence = torch.arange(0, 10, 2)
print(f"\nSequence (0 to 10, step 2): {sequence}")

# Like another tensor
like_zeros = torch.zeros_like(random_normal)
print(f"\nZeros like random_normal:\n{like_zeros}")

### Tensor Attributes

Every tensor has three key attributes:

| Attribute | What it tells you | Example |
|-----------|-------------------|--------|
| **shape** | Dimensions of the tensor | `torch.Size([3, 4])` |
| **dtype** | Data type of elements | `torch.float32`, `torch.int64` |
| **device** | Where the tensor lives | `cpu` or `cuda:0` |

In [ ]:
# Tensor attributes
x = torch.randn(3, 4, 5)

print("Tensor Attributes")
print("=" * 50)
print(f"Shape: {x.shape}")       # Dimensions
print(f"Size:  {x.size()}")      # Same as shape (method form)
print(f"Dtype: {x.dtype}")       # Data type
print(f"Device: {x.device}")     # CPU or GPU
print(f"Requires grad: {x.requires_grad}")  # Tracking gradients?

# Common data types
print("\nCommon Data Types:")
print(f"Float tensor:   {torch.tensor([1.0]).dtype}")
print(f"Int tensor:     {torch.tensor([1]).dtype}")
print(f"Explicit float: {torch.tensor([1], dtype=torch.float32).dtype}")

### Tensor Operations

PyTorch operations mirror NumPy almost exactly. If you know NumPy, you know 90% of PyTorch tensor operations.

In [ ]:
# Tensor operations - same as NumPy!
a = torch.tensor([[1., 2.], [3., 4.]])
b = torch.tensor([[5., 6.], [7., 8.]])

print("Tensor Operations")
print("=" * 50)
print(f"a:\n{a}")
print(f"\nb:\n{b}")

# Element-wise operations
print(f"\na + b (element-wise):\n{a + b}")
print(f"\na * b (element-wise):\n{a * b}")
print(f"\na ** 2 (element-wise square):\n{a ** 2}")

# Matrix multiplication
print(f"\na @ b (matrix multiply):\n{a @ b}")

# Reductions
print(f"\na.sum(): {a.sum()}")
print(f"a.mean(): {a.mean()}")
print(f"a.sum(dim=0): {a.sum(dim=0)}")  # Sum along rows
print(f"a.sum(dim=1): {a.sum(dim=1)}")  # Sum along columns

# Reshaping
c = torch.arange(12)
print(f"\nOriginal: {c}")
print(f"Reshaped to 3x4:\n{c.reshape(3, 4)}")
print(f"View as 4x3:\n{c.view(4, 3)}")

### Converting Between NumPy and PyTorch

You'll often need to move data between NumPy and PyTorch:

| Direction | Method | Notes |
|-----------|--------|-------|
| NumPy -> PyTorch | `torch.from_numpy(arr)` | Shares memory! |
| PyTorch -> NumPy | `tensor.numpy()` | Shares memory (CPU only) |
| Safe copy | `torch.tensor(arr)` | Creates a copy |

In [ ]:
# NumPy <-> PyTorch conversion
print("NumPy <-> PyTorch Conversion")
print("=" * 50)

# NumPy to PyTorch (shares memory!)
np_arr = np.array([1., 2., 3.])
torch_from_np = torch.from_numpy(np_arr)
print(f"NumPy array: {np_arr}")
print(f"Torch from NumPy: {torch_from_np}")

# They share memory - modifying one affects the other!
np_arr[0] = 999
print(f"\nAfter modifying NumPy array:")
print(f"NumPy array: {np_arr}")
print(f"Torch tensor: {torch_from_np}")  # Also changed!

# PyTorch to NumPy
torch_tensor = torch.tensor([4., 5., 6.])
np_from_torch = torch_tensor.numpy()
print(f"\nTorch tensor: {torch_tensor}")
print(f"NumPy from Torch: {np_from_torch}")

# Safe copy (no shared memory)
np_safe = np.array([7., 8., 9.])
torch_copy = torch.tensor(np_safe)  # Creates a copy
np_safe[0] = 0
print(f"\nSafe copy (using torch.tensor):")
print(f"NumPy (modified): {np_safe}")
print(f"Torch (unchanged): {torch_copy}")

### GPU Tensors

One of PyTorch's killer features is seamless GPU support. Moving tensors to GPU is simple:

```python
# Move to GPU
x_gpu = x.to('cuda')        # Or x.cuda()

# Move back to CPU
x_cpu = x_gpu.to('cpu')     # Or x_gpu.cpu()

# Device-agnostic code
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = x.to(device)
```

**Note:** All tensors in an operation must be on the same device!

In [ ]:
# GPU tensors (works even without GPU - just stays on CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Create tensor on the best available device
x = torch.randn(3, 3, device=device)
print(f"\nTensor on {x.device}:\n{x}")

# Or move an existing tensor
y = torch.randn(3, 3)
y = y.to(device)
print(f"\nMoved tensor to {y.device}")

# Operations happen on the same device
z = x @ y
print(f"Result on {z.device}")

### Visualizing Tensor Shapes

Understanding tensor shapes is crucial. Let's visualize how different tensors are organized.

In [ ]:
# Visualizing tensor dimensions
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# 0D: Scalar
ax = axes[0]
ax.text(0.5, 0.5, '42', fontsize=40, ha='center', va='center',
        bbox=dict(boxstyle='circle', facecolor='lightblue', edgecolor='blue', linewidth=2))
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_title('0D Tensor (Scalar)\nshape: ()', fontsize=12)
ax.axis('off')

# 1D: Vector
ax = axes[1]
vector = [1, 2, 3, 4]
for i, v in enumerate(vector):
    ax.add_patch(plt.Rectangle((i, 0), 0.9, 0.9, facecolor='lightgreen', edgecolor='green', linewidth=2))
    ax.text(i + 0.45, 0.45, str(v), ha='center', va='center', fontsize=14)
ax.set_xlim(-0.2, 4.2)
ax.set_ylim(-0.2, 1.2)
ax.set_title('1D Tensor (Vector)\nshape: (4,)', fontsize=12)
ax.axis('off')

# 2D: Matrix
ax = axes[2]
matrix = [[1, 2, 3], [4, 5, 6]]
for i, row in enumerate(matrix):
    for j, v in enumerate(row):
        ax.add_patch(plt.Rectangle((j, 1-i), 0.9, 0.9, facecolor='lightyellow', edgecolor='orange', linewidth=2))
        ax.text(j + 0.45, 1.45 - i, str(v), ha='center', va='center', fontsize=14)
ax.set_xlim(-0.2, 3.2)
ax.set_ylim(-0.2, 2.2)
ax.set_title('2D Tensor (Matrix)\nshape: (2, 3)', fontsize=12)
ax.axis('off')

# 3D: Batch of matrices
ax = axes[3]
colors = ['lightcoral', 'lightblue']
offsets = [(0, 0), (0.4, 0.4)]
for batch, (ox, oy) in enumerate(offsets):
    for i in range(2):
        for j in range(2):
            ax.add_patch(plt.Rectangle((j + ox, 1 - i + oy), 0.9, 0.9, 
                                       facecolor=colors[batch], edgecolor='gray', linewidth=1, alpha=0.8))
ax.set_xlim(-0.2, 3)
ax.set_ylim(-0.2, 3)
ax.set_title('3D Tensor\nshape: (2, 2, 2)', fontsize=12)
ax.axis('off')

plt.tight_layout()
plt.savefig('tensor_shapes.png', dpi=150, bbox_inches='tight')
plt.show()

print("Common tensor shapes in deep learning:")
print("  (batch, features)          - Tabular data")
print("  (batch, channels, H, W)    - Images")
print("  (batch, seq_len, features) - Sequences/text")

---

## 3. Autograd: Automatic Differentiation

### The Magic of Automatic Gradients

Remember computing gradients by hand in your from-scratch implementations? You had to:
1. Derive the gradient formula for each operation
2. Implement the backward pass manually
3. Chain rule everything together

**Autograd does all of this automatically!**

### How It Works (Intuition)

When you set `requires_grad=True` on a tensor, PyTorch:
1. **Records** every operation you perform on that tensor
2. **Builds** a computation graph in the background
3. **Computes** gradients automatically when you call `.backward()`

It's like having a perfect calculus student watching everything you do and computing derivatives on demand.

In [ ]:
# Basic autograd example
print("Basic Autograd Example")
print("=" * 50)

# Create a tensor with gradient tracking
x = torch.tensor([2.0], requires_grad=True)
print(f"x = {x.item()}, requires_grad = {x.requires_grad}")

# Perform some operations: y = x^2 + 3x + 1
y = x**2 + 3*x + 1
print(f"y = x^2 + 3x + 1 = {y.item()}")

# Compute gradients: dy/dx = 2x + 3 = 2(2) + 3 = 7
y.backward()

print(f"\nGradient dy/dx at x=2: {x.grad.item()}")
print(f"Manual calculation: 2*x + 3 = 2*2 + 3 = 7 ✓")

### The Three Key Concepts

| Concept | Code | What it does |
|---------|------|-------------|
| **Enable tracking** | `requires_grad=True` | Tells PyTorch to record operations |
| **Compute gradients** | `.backward()` | Runs backpropagation |
| **Access gradients** | `.grad` | Gets the computed gradient |

In [ ]:
# More complex example with multiple variables
print("Autograd with Multiple Variables")
print("=" * 50)

# Two input variables
a = torch.tensor([3.0], requires_grad=True)
b = torch.tensor([4.0], requires_grad=True)

# Complex expression: c = a^2 * b + sin(b)
c = a**2 * b + torch.sin(b)
print(f"a = {a.item()}, b = {b.item()}")
print(f"c = a^2 * b + sin(b) = {c.item():.4f}")

# Compute gradients
c.backward()

# dc/da = 2*a*b = 2*3*4 = 24
print(f"\ndc/da = {a.grad.item():.4f}")
print(f"Manual: 2*a*b = 2*3*4 = 24.0 ✓")

# dc/db = a^2 + cos(b) = 9 + cos(4)
print(f"\ndc/db = {b.grad.item():.4f}")
print(f"Manual: a^2 + cos(b) = 9 + cos(4) = {9 + np.cos(4):.4f} ✓")

### Deep Dive: How Autograd Works Under the Hood

PyTorch builds a **dynamic computation graph** as you perform operations. Each node in the graph knows:
1. What operation created it
2. What its inputs were
3. How to compute its local gradient

When you call `.backward()`, PyTorch walks backward through this graph, applying the chain rule automatically.

#### Key Insight

The graph is **dynamic** - it's rebuilt fresh each forward pass. This is different from TensorFlow 1.x where graphs were static. Dynamic graphs are:
- Easier to debug (use regular Python debugger)
- More flexible (control flow just works)
- More intuitive (code runs line by line)

#### Common Misconceptions

| Misconception | Reality |
|---------------|--------|
| "Autograd is magic" | It's just the chain rule applied systematically |
| "Gradients accumulate by default" | Yes! Call `.zero_()` before each backward pass |
| "I need to track all tensors" | Only track tensors you want to differentiate w.r.t. |

In [ ]:
# Visualizing the computation graph
fig, ax = plt.subplots(figsize=(12, 8))

# Draw computation graph for y = (a*x + b)^2
nodes = {
    'x': (1, 4),
    'a': (1, 2),
    'b': (3, 2),
    'ax': (2, 3),
    'ax+b': (3, 4),
    'y=(ax+b)^2': (4, 3)
}

edges = [
    ('x', 'ax', 'x'),
    ('a', 'ax', 'a'),
    ('ax', 'ax+b', ''),
    ('b', 'ax+b', 'b'),
    ('ax+b', 'y=(ax+b)^2', '')
]

# Draw nodes
for name, (x, y) in nodes.items():
    color = 'lightgreen' if name in ['x', 'a', 'b'] else 'lightblue'
    ax.add_patch(plt.Circle((x, y), 0.3, facecolor=color, edgecolor='black', linewidth=2))
    ax.text(x, y, name, ha='center', va='center', fontsize=9, fontweight='bold')

# Draw edges with gradient labels
gradient_labels = {
    ('x', 'ax'): 'dy/dx = a*2(ax+b)',
    ('a', 'ax'): 'dy/da = x*2(ax+b)',
    ('b', 'ax+b'): 'dy/db = 2(ax+b)'
}

for start, end, _ in edges:
    x1, y1 = nodes[start]
    x2, y2 = nodes[end]
    ax.annotate('', xy=(x2, y2), xytext=(x1, y1),
                arrowprops=dict(arrowstyle='->', color='gray', lw=2))

# Add gradient flow arrows (backward direction)
ax.annotate('', xy=(3.3, 3.8), xytext=(3.7, 3.3),
            arrowprops=dict(arrowstyle='->', color='red', lw=2))
ax.text(3.7, 3.5, 'gradient\nflows back', color='red', fontsize=10, ha='left')

ax.set_xlim(0, 5.5)
ax.set_ylim(1, 5)
ax.set_title('Computation Graph: y = (ax + b)^2\n(Gradients flow backward through the graph)', fontsize=14)
ax.axis('off')

# Add legend
ax.add_patch(plt.Circle((0.5, 1.5), 0.15, facecolor='lightgreen', edgecolor='black'))
ax.text(0.8, 1.5, 'Input (leaf) tensors', va='center', fontsize=10)
ax.add_patch(plt.Circle((0.5, 1.2), 0.15, facecolor='lightblue', edgecolor='black'))
ax.text(0.8, 1.2, 'Intermediate values', va='center', fontsize=10)

plt.tight_layout()
plt.savefig('computation_graph.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Gradient accumulation demo - a common gotcha!
print("Gradient Accumulation (Important!)")
print("=" * 50)

w = torch.tensor([1.0], requires_grad=True)

# First backward pass
y1 = w * 2
y1.backward()
print(f"After first backward: w.grad = {w.grad.item()}")

# Second backward pass WITHOUT zeroing gradients
y2 = w * 2
y2.backward()
print(f"After second backward (accumulated!): w.grad = {w.grad.item()}")

# The right way: zero gradients first
w.grad.zero_()  # Reset to zero
y3 = w * 2
y3.backward()
print(f"After zeroing and backward: w.grad = {w.grad.item()}")

print("\n⚠️ Always zero gradients before each training step!")

### Disabling Gradient Tracking

During inference (testing), you don't need gradients. Disabling them:
- Saves memory
- Speeds up computation

Two ways to disable:

| Method | Use case |
|--------|----------|
| `with torch.no_grad():` | Temporarily disable (inference) |
| `.detach()` | Remove tensor from graph permanently |

In [ ]:
# Disabling gradients
print("Disabling Gradient Tracking")
print("=" * 50)

x = torch.tensor([3.0], requires_grad=True)

# Normal: gradients tracked
y = x * 2
print(f"Normal: y.requires_grad = {y.requires_grad}")

# With no_grad context: no tracking
with torch.no_grad():
    y_no_grad = x * 2
    print(f"In no_grad: y.requires_grad = {y_no_grad.requires_grad}")

# Back to normal after the context
y_after = x * 2
print(f"After no_grad: y.requires_grad = {y_after.requires_grad}")

# Detach: permanently removes from graph
y_detached = y.detach()
print(f"\nDetached: y_detached.requires_grad = {y_detached.requires_grad}")

print("\n💡 Use torch.no_grad() during inference to save memory!")

### Autograd Example: Gradient Descent from Scratch

Let's use autograd to minimize a function, just like we did by hand before!

In [ ]:
# Gradient descent using autograd
print("Gradient Descent with Autograd")
print("=" * 50)

# Minimize f(x) = (x - 3)^2 + 1
# Minimum at x = 3

x = torch.tensor([0.0], requires_grad=True)
learning_rate = 0.1
history = {'x': [], 'loss': []}

for step in range(50):
    # Forward pass: compute loss
    loss = (x - 3)**2 + 1
    
    # Record history
    history['x'].append(x.item())
    history['loss'].append(loss.item())
    
    # Backward pass: compute gradients
    loss.backward()
    
    # Update x using gradient descent
    with torch.no_grad():  # Don't track this operation
        x -= learning_rate * x.grad
    
    # Zero gradient for next iteration
    x.grad.zero_()
    
    if step % 10 == 0:
        print(f"Step {step:2d}: x = {history['x'][-1]:.4f}, loss = {history['loss'][-1]:.4f}")

print(f"\nFinal x = {x.item():.4f} (should be close to 3.0)")

In [ ]:
# Visualize the gradient descent
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Function and optimization path
ax = axes[0]
x_range = np.linspace(-1, 5, 100)
y_range = (x_range - 3)**2 + 1
ax.plot(x_range, y_range, 'b-', linewidth=2, label='f(x) = (x-3)^2 + 1')
ax.scatter(history['x'], history['loss'], c=range(len(history['x'])), 
           cmap='Reds', s=50, zorder=5, label='Gradient descent path')
ax.scatter([3], [1], color='green', s=200, marker='*', zorder=6, label='Minimum')
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.set_title('Gradient Descent Finding the Minimum')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: Loss over time
ax = axes[1]
ax.plot(history['loss'], 'b-', linewidth=2)
ax.set_xlabel('Step')
ax.set_ylabel('Loss')
ax.set_title('Loss Decreasing Over Time')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('autograd_gd.png', dpi=150, bbox_inches='tight')
plt.show()

---

## 4. nn.Module: Building Blocks

### Why Use nn.Module?

You could build neural networks using raw tensors and autograd. But `nn.Module` provides:

| Feature | Benefit |
|---------|--------|
| **Parameter management** | Automatically tracks all learnable weights |
| **GPU movement** | `.to(device)` moves all parameters at once |
| **Training/eval modes** | `.train()` / `.eval()` for dropout, batchnorm |
| **Hooks** | Inspect activations, gradients during forward/backward |
| **Serialization** | Easy save/load with state_dict |

### The Basic Pattern

Every neural network module follows the same pattern:

```python
class MyModule(nn.Module):
    def __init__(self):
        super().__init__()
        # Define layers here
        self.layer1 = nn.Linear(10, 5)
        
    def forward(self, x):
        # Define computation here
        return self.layer1(x)
```

In [ ]:
# Building a simple custom module
print("Building Custom Modules")
print("=" * 50)

class SimpleLinear(nn.Module):
    """A simple linear layer: y = Wx + b (what you built from scratch!)"""
    
    def __init__(self, in_features, out_features):
        super().__init__()
        # nn.Parameter tells PyTorch these are learnable
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.bias = nn.Parameter(torch.zeros(out_features))
        
    def forward(self, x):
        # Same computation as your from-scratch implementation!
        return x @ self.weight.T + self.bias

# Create and test
layer = SimpleLinear(3, 2)
x = torch.randn(5, 3)  # 5 samples, 3 features
output = layer(x)      # Calls forward() automatically

print(f"Input shape:  {x.shape}")
print(f"Output shape: {output.shape}")
print(f"\nWeight shape: {layer.weight.shape}")
print(f"Bias shape:   {layer.bias.shape}")

# List all parameters
print("\nAll parameters:")
for name, param in layer.named_parameters():
    print(f"  {name}: {param.shape}")

### Common Built-in Layers

PyTorch provides optimized versions of common layers:

| Layer | Purpose | Parameters |
|-------|---------|------------|
| `nn.Linear(in, out)` | Fully connected | W: (out, in), b: (out,) |
| `nn.Conv2d(in_ch, out_ch, kernel)` | 2D convolution | W: (out_ch, in_ch, k, k) |
| `nn.ReLU()` | Activation | None |
| `nn.Sigmoid()` | Activation | None |
| `nn.Dropout(p)` | Regularization | None |
| `nn.BatchNorm1d(features)` | Normalization | gamma, beta |
| `nn.Embedding(vocab, dim)` | Word embeddings | (vocab, dim) |

In [ ]:
# Exploring common layers
print("Common PyTorch Layers")
print("=" * 50)

# Linear layer
linear = nn.Linear(10, 5)
x = torch.randn(32, 10)  # Batch of 32
print(f"nn.Linear(10, 5):")
print(f"  Input:  {x.shape}")
print(f"  Output: {linear(x).shape}")
print(f"  Params: weight {linear.weight.shape}, bias {linear.bias.shape}")

# ReLU activation
relu = nn.ReLU()
x = torch.tensor([-2., -1., 0., 1., 2.])
print(f"\nnn.ReLU():")
print(f"  Input:  {x}")
print(f"  Output: {relu(x)}")

# Dropout (training mode)
dropout = nn.Dropout(p=0.5)  # 50% dropout
x = torch.ones(10)
print(f"\nnn.Dropout(0.5) in training mode:")
dropout.train()  # Enable dropout
print(f"  Input:  {x}")
print(f"  Output: {dropout(x)}")  # Some values zeroed and others scaled
dropout.eval()  # Disable dropout
print(f"  In eval mode: {dropout(x)}")  # No dropout applied

### nn.Sequential: Quick Model Building

For simple feed-forward networks, `nn.Sequential` lets you stack layers without writing a class:

In [ ]:
# nn.Sequential for quick models
print("nn.Sequential Example")
print("=" * 50)

# A simple MLP using Sequential
mlp = nn.Sequential(
    nn.Linear(784, 256),   # Input layer
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(256, 128),   # Hidden layer
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 10)     # Output layer
)

print(mlp)

# Test it
x = torch.randn(32, 784)  # Batch of 32 flattened MNIST images
output = mlp(x)
print(f"\nInput shape:  {x.shape}")
print(f"Output shape: {output.shape}")

# Count parameters
total_params = sum(p.numel() for p in mlp.parameters())
print(f"\nTotal parameters: {total_params:,}")

---

## 5. Building a Complete Model

Let's build a proper neural network class that combines everything:
- Multiple layers
- Activations
- Clear forward method

This is equivalent to what you built from scratch, but now PyTorch handles all the gradient computation!

In [ ]:
# A complete neural network model
class NeuralNetwork(nn.Module):
    """
    A flexible feed-forward neural network.
    
    This is the PyTorch equivalent of the network you built from scratch!
    """
    
    def __init__(self, input_dim, hidden_dims, output_dim):
        """
        Args:
            input_dim: Number of input features
            hidden_dims: List of hidden layer sizes
            output_dim: Number of output classes/values
        """
        super().__init__()
        
        # Build layers dynamically
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.2))
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, output_dim))
        
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        """
        Forward pass through the network.
        
        Args:
            x: Input tensor of shape (batch_size, input_dim)
            
        Returns:
            Output tensor of shape (batch_size, output_dim)
        """
        return self.network(x)
    
    def count_parameters(self):
        """Count total trainable parameters."""
        return sum(p.numel() for p in self.parameters() if p.requires_grad)


# Create the model
model = NeuralNetwork(
    input_dim=784,      # 28x28 flattened image
    hidden_dims=[256, 128, 64],  # Three hidden layers
    output_dim=10       # 10 digit classes
)

print("Model Architecture:")
print("=" * 50)
print(model)
print(f"\nTotal parameters: {model.count_parameters():,}")

In [ ]:
# Test the model
print("\nTesting the model:")
print("=" * 50)

# Create dummy input
batch_size = 32
x = torch.randn(batch_size, 784)

# Forward pass
model.eval()  # Set to evaluation mode
with torch.no_grad():
    output = model(x)

print(f"Input shape:  {x.shape}")
print(f"Output shape: {output.shape}")

# Get predictions
probabilities = F.softmax(output, dim=1)
predictions = torch.argmax(probabilities, dim=1)

print(f"\nSample output (raw logits): {output[0][:5]}...")  # First 5 logits
print(f"Sample probabilities: {probabilities[0][:5]}...")
print(f"Predicted class: {predictions[0].item()}")

### Visualizing Model Architecture

In [ ]:
# Visualize the network architecture
fig, ax = plt.subplots(figsize=(14, 8))

# Layer sizes for our model
layer_sizes = [784, 256, 128, 64, 10]
layer_names = ['Input\n(784)', 'Hidden 1\n(256)', 'Hidden 2\n(128)', 'Hidden 3\n(64)', 'Output\n(10)']

# Normalize for visualization (show fewer neurons)
max_neurons_display = 8
display_sizes = [min(s, max_neurons_display) for s in layer_sizes]

# Draw layers
x_positions = np.linspace(1, 9, len(layer_sizes))
colors = ['lightgreen', 'lightblue', 'lightblue', 'lightblue', 'lightyellow']

for layer_idx, (x_pos, n_neurons, color, name) in enumerate(zip(x_positions, display_sizes, colors, layer_names)):
    y_positions = np.linspace(1, 7, n_neurons)
    
    for y_pos in y_positions:
        circle = plt.Circle((x_pos, y_pos), 0.2, facecolor=color, edgecolor='black', linewidth=1.5)
        ax.add_patch(circle)
    
    # Add "..." if we're not showing all neurons
    if layer_sizes[layer_idx] > max_neurons_display:
        ax.text(x_pos, 0.4, '...', ha='center', va='center', fontsize=16)
    
    # Layer name
    ax.text(x_pos, 8, name, ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    # Draw connections to next layer (simplified)
    if layer_idx < len(layer_sizes) - 1:
        next_x = x_positions[layer_idx + 1]
        next_neurons = display_sizes[layer_idx + 1]
        next_y_positions = np.linspace(1, 7, next_neurons)
        
        # Draw a subset of connections
        for i, y1 in enumerate(y_positions[::2]):  # Every other neuron
            for j, y2 in enumerate(next_y_positions[::2]):
                ax.plot([x_pos + 0.2, next_x - 0.2], [y1, y2], 'gray', alpha=0.2, linewidth=0.5)

# Add ReLU labels
for i in range(3):
    x_mid = (x_positions[i] + x_positions[i+1]) / 2
    ax.text(x_mid, 0, 'ReLU', ha='center', va='center', fontsize=9,
            bbox=dict(boxstyle='round', facecolor='white', edgecolor='gray'))

ax.set_xlim(0, 10)
ax.set_ylim(-0.5, 9)
ax.set_title('Neural Network Architecture\n(784 -> 256 -> 128 -> 64 -> 10)', fontsize=14)
ax.axis('off')

plt.tight_layout()
plt.savefig('network_architecture.png', dpi=150, bbox_inches='tight')
plt.show()

---

## 6. Training Loop Anatomy

### The Core Training Loop

Every PyTorch training loop follows the same pattern:

```python
for epoch in range(num_epochs):
    for batch in dataloader:
        # 1. Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # 2. Backward pass
        optimizer.zero_grad()  # Clear old gradients
        loss.backward()        # Compute new gradients
        
        # 3. Update weights
        optimizer.step()
```

### Breaking Down Each Step

| Step | What happens | PyTorch code |
|------|--------------|-------------|
| **Forward** | Input flows through network | `outputs = model(inputs)` |
| **Loss** | Compare predictions to targets | `loss = criterion(outputs, targets)` |
| **Zero grad** | Clear accumulated gradients | `optimizer.zero_grad()` |
| **Backward** | Compute gradients via backprop | `loss.backward()` |
| **Step** | Update weights using gradients | `optimizer.step()` |

In [ ]:
# Visualize the training loop
fig, ax = plt.subplots(figsize=(14, 8))

# Draw training loop steps
steps = [
    ('1. Forward Pass', 'Input -> Model -> Predictions', 'lightgreen'),
    ('2. Compute Loss', 'Compare predictions to targets', 'lightyellow'),
    ('3. Zero Gradients', 'Clear old gradients', 'lightgray'),
    ('4. Backward Pass', 'Compute gradients (backprop)', 'lightcoral'),
    ('5. Optimizer Step', 'Update weights', 'lightblue'),
]

y_positions = np.linspace(7, 1, len(steps))
box_width = 4
box_height = 0.8

for i, (title, desc, color) in enumerate(steps):
    y = y_positions[i]
    
    # Main box
    rect = plt.Rectangle((1, y - box_height/2), box_width, box_height,
                         facecolor=color, edgecolor='black', linewidth=2)
    ax.add_patch(rect)
    ax.text(3, y, title, ha='center', va='center', fontsize=12, fontweight='bold')
    
    # Description
    ax.text(5.5, y, desc, ha='left', va='center', fontsize=11)
    
    # Arrow to next step
    if i < len(steps) - 1:
        ax.annotate('', xy=(3, y_positions[i+1] + box_height/2 + 0.1),
                   xytext=(3, y - box_height/2 - 0.1),
                   arrowprops=dict(arrowstyle='->', color='black', lw=2))

# Add loop arrow
ax.annotate('', xy=(0.5, y_positions[0]),
           xytext=(0.5, y_positions[-1]),
           arrowprops=dict(arrowstyle='->', color='blue', lw=2,
                          connectionstyle='arc3,rad=0.3'))
ax.text(0.2, 4, 'Repeat\nfor each\nbatch', ha='center', va='center', fontsize=10, color='blue')

ax.set_xlim(-0.5, 10)
ax.set_ylim(0, 8.5)
ax.set_title('The PyTorch Training Loop', fontsize=14, fontweight='bold')
ax.axis('off')

plt.tight_layout()
plt.savefig('training_loop.png', dpi=150, bbox_inches='tight')
plt.show()

### Complete Training Example on Synthetic Data

Let's train a model on synthetic data to see the full loop in action.

In [ ]:
# Generate synthetic dataset
print("Creating Synthetic Dataset")
print("=" * 50)

# Binary classification: predict if x is inside or outside a circle
n_samples = 1000

# Generate random points
X = np.random.randn(n_samples, 2) * 2

# Labels: 1 if inside circle of radius 1.5, else 0
distances = np.sqrt(X[:, 0]**2 + X[:, 1]**2)
y = (distances < 1.5).astype(np.float32)

# Convert to tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

# Split into train/test
train_size = int(0.8 * n_samples)
X_train, X_test = X_tensor[:train_size], X_tensor[train_size:]
y_train, y_test = y_tensor[:train_size], y_tensor[train_size:]

print(f"Training samples: {len(X_train)}")
print(f"Test samples:     {len(X_test)}")
print(f"Input features:   {X_train.shape[1]}")
print(f"Class balance:    {y_train.mean().item():.2%} positive")

In [ ]:
# Visualize the dataset
fig, ax = plt.subplots(figsize=(8, 8))

colors = ['red' if label == 0 else 'blue' for label in y]
ax.scatter(X[:, 0], X[:, 1], c=colors, alpha=0.5, s=20)

# Draw the decision boundary (circle)
theta = np.linspace(0, 2*np.pi, 100)
ax.plot(1.5 * np.cos(theta), 1.5 * np.sin(theta), 'g--', linewidth=2, label='True boundary')

ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.set_title('Synthetic Dataset: Points Inside (blue) vs Outside (red) Circle')
ax.legend()
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('synthetic_data.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Define the model for our binary classification task
class BinaryClassifier(nn.Module):
    """Simple binary classifier for our circle dataset."""
    
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(2, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()  # Output probability
        )
    
    def forward(self, x):
        return self.network(x)

# Create model, loss function, and optimizer
model = BinaryClassifier()
criterion = nn.BCELoss()  # Binary Cross Entropy
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

print("Model:")
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# THE TRAINING LOOP
print("\nTraining Loop")
print("=" * 50)

num_epochs = 100
batch_size = 64
n_batches = len(X_train) // batch_size

# History for plotting
history = {'train_loss': [], 'test_loss': [], 'test_acc': []}

for epoch in range(num_epochs):
    model.train()  # Set to training mode
    epoch_loss = 0
    
    # Shuffle data
    perm = torch.randperm(len(X_train))
    X_shuffled = X_train[perm]
    y_shuffled = y_train[perm]
    
    for i in range(n_batches):
        # Get batch
        start = i * batch_size
        end = start + batch_size
        X_batch = X_shuffled[start:end]
        y_batch = y_shuffled[start:end]
        
        # ====== THE FIVE KEY STEPS ======
        
        # 1. Forward pass
        predictions = model(X_batch)
        
        # 2. Compute loss
        loss = criterion(predictions, y_batch)
        
        # 3. Zero gradients
        optimizer.zero_grad()
        
        # 4. Backward pass
        loss.backward()
        
        # 5. Update weights
        optimizer.step()
        
        # ================================
        
        epoch_loss += loss.item()
    
    # Evaluate on test set
    model.eval()  # Set to evaluation mode
    with torch.no_grad():
        test_pred = model(X_test)
        test_loss = criterion(test_pred, y_test).item()
        test_acc = ((test_pred > 0.5) == y_test).float().mean().item()
    
    # Record history
    history['train_loss'].append(epoch_loss / n_batches)
    history['test_loss'].append(test_loss)
    history['test_acc'].append(test_acc)
    
    if epoch % 20 == 0 or epoch == num_epochs - 1:
        print(f"Epoch {epoch:3d}: Train Loss = {epoch_loss/n_batches:.4f}, "
              f"Test Loss = {test_loss:.4f}, Test Acc = {test_acc:.2%}")

print("\nTraining complete!")

In [ ]:
# Visualize training progress
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Loss curves
ax = axes[0]
ax.plot(history['train_loss'], label='Train Loss', color='blue')
ax.plot(history['test_loss'], label='Test Loss', color='red')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Training and Test Loss')
ax.legend()
ax.grid(True, alpha=0.3)

# Accuracy
ax = axes[1]
ax.plot(history['test_acc'], color='green')
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax.set_title('Test Accuracy')
ax.set_ylim(0, 1.05)
ax.grid(True, alpha=0.3)

# Decision boundary
ax = axes[2]

# Create a mesh grid
x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                      np.linspace(y_min, y_max, 100))

# Get predictions on the grid
grid_tensor = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)
model.eval()
with torch.no_grad():
    Z = model(grid_tensor).numpy().reshape(xx.shape)

# Plot decision boundary
ax.contourf(xx, yy, Z, levels=50, cmap='RdBu', alpha=0.6)
ax.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2)

# Plot test points
colors_test = ['red' if label == 0 else 'blue' for label in y_test.numpy()]
ax.scatter(X_test[:, 0], X_test[:, 1], c=colors_test, alpha=0.7, s=30, edgecolors='white', linewidth=0.5)

# True boundary
theta = np.linspace(0, 2*np.pi, 100)
ax.plot(1.5 * np.cos(theta), 1.5 * np.sin(theta), 'g--', linewidth=2, label='True boundary')

ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.set_title('Learned Decision Boundary')
ax.legend(loc='upper right')
ax.set_aspect('equal')

plt.tight_layout()
plt.savefig('training_results.png', dpi=150, bbox_inches='tight')
plt.show()

### Connection to Your From-Scratch Implementation

Compare what you did manually vs what PyTorch does:

| From Scratch | PyTorch | Notes |
|--------------|---------|-------|
| Manually derived gradients | `loss.backward()` | Autograd computes automatically |
| Stored intermediate values | Built into computation graph | PyTorch handles this |
| `weights -= lr * gradients` | `optimizer.step()` | Optimizer handles updates |
| Implemented ReLU, Sigmoid | `nn.ReLU()`, `nn.Sigmoid()` | Pre-built, optimized |
| Tracked loss manually | Same | You still do this |

**Key insight:** The conceptual steps are identical! PyTorch just automates the tedious parts.

---

## 7. Saving and Loading Models

After training, you'll want to save your model for later use. PyTorch offers two approaches:

| Method | What it saves | Use case |
|--------|--------------|----------|
| `state_dict()` | Just the weights | Recommended for most cases |
| `torch.save(model)` | Entire model | Convenient but less flexible |

### Best Practice: Save state_dict()

The `state_dict()` contains all learnable parameters as a dictionary:

In [ ]:
# Examining state_dict
print("Model State Dict")
print("=" * 50)

state_dict = model.state_dict()

for name, tensor in state_dict.items():
    print(f"{name}: {tensor.shape}")

print(f"\nTotal parameters: {sum(t.numel() for t in state_dict.values()):,}")

In [ ]:
# Saving and loading
print("\nSaving and Loading Models")
print("=" * 50)

# Save
save_path = 'circle_classifier.pth'
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}")

# Load into a new model
new_model = BinaryClassifier()  # Create same architecture
new_model.load_state_dict(torch.load(save_path, weights_only=True))
new_model.eval()

print("Model loaded successfully!")

# Verify it works
with torch.no_grad():
    original_pred = model(X_test[:5])
    loaded_pred = new_model(X_test[:5])

print(f"\nVerification (predictions should match):")
print(f"Original: {original_pred.flatten().numpy()}")
print(f"Loaded:   {loaded_pred.flatten().numpy()}")
print(f"Match: {torch.allclose(original_pred, loaded_pred)}")

In [ ]:
# Saving checkpoints with more information
print("\nSaving Complete Checkpoint")
print("=" * 50)

checkpoint = {
    'epoch': num_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_loss': history['train_loss'][-1],
    'test_acc': history['test_acc'][-1],
}

torch.save(checkpoint, 'checkpoint.pth')
print("Checkpoint saved with:")
for key in checkpoint.keys():
    print(f"  - {key}")

# Loading checkpoint
print("\nLoading checkpoint:")
loaded_checkpoint = torch.load('checkpoint.pth', weights_only=False)
print(f"  Trained for {loaded_checkpoint['epoch']} epochs")
print(f"  Final test accuracy: {loaded_checkpoint['test_acc']:.2%}")

### Deep Dive: state_dict vs Pickling the Model

You might be tempted to just save the entire model:

```python
# This works but has drawbacks
torch.save(model, 'model.pth')
model = torch.load('model.pth')
```

Why `state_dict()` is better:

| Issue | Saving entire model | Saving state_dict |
|-------|---------------------|------------------|
| **File includes** | Code + weights | Just weights |
| **Code changes** | Breaks loading | No problem |
| **Different PyTorch version** | May break | Usually works |
| **Flexibility** | Fixed architecture | Load into any compatible architecture |

---

## Exercises

### Exercise 1: Tensor Warmup

Create and manipulate tensors to get comfortable with the PyTorch API.

In [ ]:
# EXERCISE 1: Tensor Warmup

def tensor_exercises():
    """
    Complete the following tensor operations:
    
    1. Create a 3x4 tensor of ones
    2. Create a 4x5 tensor of random values (normal distribution)
    3. Multiply them together (matrix multiplication)
    4. Take the sum along dimension 1
    5. Return the result
    """
    # TODO: Implement this!
    # Hint: Use torch.ones(), torch.randn(), @, and .sum()
    
    # Step 1: Create ones tensor
    a = None  # Your code here
    
    # Step 2: Create random tensor
    b = None  # Your code here
    
    # Step 3: Matrix multiplication
    c = None  # Your code here
    
    # Step 4: Sum along dimension 1
    result = None  # Your code here
    
    return result

# Test
torch.manual_seed(42)
result = tensor_exercises()
print(f"Your result shape: {result.shape if result is not None else 'None'}")
print(f"Expected shape: torch.Size([3])")
print(f"Your result: {result}")

### Exercise 2: Manual Gradient Verification

Use autograd to verify your calculus skills!

In [ ]:
# EXERCISE 2: Gradient Verification

def verify_gradient(x_value, function_str):
    """
    Given a function string, compute both:
    1. The autograd gradient
    2. Your manual calculation
    
    Function: f(x) = x^3 - 2x^2 + 3x - 1
    Derivative: f'(x) = 3x^2 - 4x + 3
    
    Return: (autograd_gradient, manual_gradient)
    """
    # Autograd computation
    x = torch.tensor([x_value], requires_grad=True, dtype=torch.float32)
    
    # f(x) = x^3 - 2x^2 + 3x - 1
    f = x**3 - 2*x**2 + 3*x - 1
    f.backward()
    autograd_gradient = x.grad.item()
    
    # TODO: Calculate the derivative manually
    # f'(x) = 3x^2 - 4x + 3
    manual_gradient = None  # Your code here
    
    return autograd_gradient, manual_gradient

# Test at x = 2
auto_grad, manual_grad = verify_gradient(2.0, "x^3 - 2x^2 + 3x - 1")
print(f"At x = 2:")
print(f"  Autograd: {auto_grad}")
print(f"  Manual:   {manual_grad}")
print(f"  Match:    {np.isclose(auto_grad, manual_grad) if manual_grad else 'Not implemented'}")

# Test at x = -1
auto_grad, manual_grad = verify_gradient(-1.0, "x^3 - 2x^2 + 3x - 1")
print(f"\nAt x = -1:")
print(f"  Autograd: {auto_grad}")
print(f"  Manual:   {manual_grad}")
print(f"  Match:    {np.isclose(auto_grad, manual_grad) if manual_grad else 'Not implemented'}")

### Exercise 3: Build a Custom Module

Implement a module that applies a custom transformation.

In [ ]:
# EXERCISE 3: Custom Module

class Swish(nn.Module):
    """
    Implement the Swish activation function:
    swish(x) = x * sigmoid(x)
    
    This is a smooth approximation to ReLU that often performs better!
    """
    
    def __init__(self):
        super().__init__()
        # No learnable parameters needed
    
    def forward(self, x):
        # TODO: Implement swish(x) = x * sigmoid(x)
        # Hint: Use torch.sigmoid()
        pass

# Test
swish = Swish()
x = torch.tensor([-2., -1., 0., 1., 2.])
output = swish(x)

print(f"Input:  {x}")
print(f"Output: {output}")

# Expected: approximately tensor([-0.2384, -0.2689, 0.0000, 0.7311, 1.7616])
expected = x * torch.sigmoid(x)
print(f"Expected: {expected}")
print(f"Correct: {torch.allclose(output, expected) if output is not None else 'Not implemented'}")

### Exercise 4: Complete Training Loop

Build and train a model for a regression task.

In [ ]:
# EXERCISE 4: Regression Training Loop

# Generate regression data: y = 2*x + 1 + noise
torch.manual_seed(42)
X_reg = torch.randn(200, 1) * 2
y_reg = 2 * X_reg + 1 + torch.randn(200, 1) * 0.5

# TODO: Complete this training loop

# 1. Create a simple linear model
# Hint: nn.Linear(1, 1) is enough for this!
reg_model = None  # Your code here

# 2. Create MSE loss function
# Hint: nn.MSELoss()
criterion = None  # Your code here

# 3. Create SGD optimizer with lr=0.01
# Hint: torch.optim.SGD(model.parameters(), lr=0.01)
optimizer = None  # Your code here

# 4. Training loop (fill in the 5 key steps)
if reg_model and criterion and optimizer:
    losses = []
    
    for epoch in range(100):
        # Forward pass
        predictions = None  # Your code
        
        # Compute loss
        loss = None  # Your code
        
        # Zero gradients
        # Your code
        
        # Backward pass
        # Your code
        
        # Update weights
        # Your code
        
        if loss:
            losses.append(loss.item())
        
        if epoch % 20 == 0 and loss:
            print(f"Epoch {epoch}: Loss = {loss.item():.4f}")
    
    # Check learned parameters
    with torch.no_grad():
        learned_weight = reg_model.weight.item()
        learned_bias = reg_model.bias.item()
        print(f"\nLearned: y = {learned_weight:.2f}*x + {learned_bias:.2f}")
        print(f"True:    y = 2.00*x + 1.00")
else:
    print("Please implement the model, criterion, and optimizer first!")

---

## Summary

### Key Concepts

- **Tensors** are PyTorch's version of NumPy arrays, with GPU support and automatic differentiation
- **Autograd** automatically computes gradients by building a computation graph during the forward pass
- **nn.Module** provides structure for organizing neural network components with automatic parameter tracking
- **The training loop** has five key steps: forward, loss, zero_grad, backward, step
- **state_dict()** is the recommended way to save and load model weights

### Connection to Deep Learning

| Concept | What you did from scratch | What PyTorch does |
|---------|--------------------------|-------------------|
| Forward pass | Implemented layer-by-layer | Same, but in `forward()` method |
| Backpropagation | Derived and coded all gradients | `loss.backward()` does it automatically |
| Weight updates | `w -= lr * grad` | `optimizer.step()` |
| Layer operations | Wrote matrix multiplications | `nn.Linear`, `nn.Conv2d`, etc. |
| Gradient tracking | Stored activations manually | Autograd tracks everything |

### Checklist

After completing this notebook, you should be able to:

- [ ] Create tensors using various methods (zeros, ones, randn, from data)
- [ ] Move tensors between CPU and GPU (conceptually)
- [ ] Convert between NumPy arrays and PyTorch tensors
- [ ] Use `requires_grad=True` to enable gradient tracking
- [ ] Call `.backward()` to compute gradients
- [ ] Access gradients via `.grad` attribute
- [ ] Use `torch.no_grad()` for inference
- [ ] Build custom modules by subclassing `nn.Module`
- [ ] Use `nn.Sequential` for simple architectures
- [ ] Write a complete training loop with the five key steps
- [ ] Save and load model weights using `state_dict()`

---

## Next Steps

Now that you understand PyTorch fundamentals, you're ready to:

1. **Part 3.4: Training on Real Data** - Use DataLoader, work with real datasets, and implement proper training/validation splits

2. **Part 4: Convolutional Neural Networks** - Apply what you've learned to image data with specialized architectures

3. **Part 5: Recurrent Neural Networks** - Process sequential data like text and time series

The foundation you've built - both from-scratch understanding and PyTorch skills - will serve you well as you tackle increasingly complex architectures!

In [ ]:
# Clean up saved files
import os
for f in ['circle_classifier.pth', 'checkpoint.pth']:
    if os.path.exists(f):
        os.remove(f)
        print(f"Removed {f}")